In [121]:
def readInput12(infile):
    with open(infile) as f:
        sources = []
        for l in f.readlines():
            e = l.strip("\n").split()
            # save as tuples instead of list to avoid problem with recursion 
            sources.append( (e[0], tuple([int(n) for n in e[1].split(",")]) ) )
        return sources

### Part 1

Brute-forcing...

In [127]:
import re
from more_itertools import distinct_permutations

def count_arrangements_bf(source):
    s,ns = source
    unknown = [ i for i,x in enumerate(s) if x=="?"]
    nunkn = len(unknown)
    nstot = sum(ns)
    nsvis  = len(re.findall("#",s))
    nsmis = nstot-nsvis
    tobeplaced = ["#"]*nsmis+["."]*(nunkn-nsmis)
    arrangements = 0 
    for p in distinct_permutations(tobeplaced):
        snew = list(s)
        for c,i in zip(p,unknown):
            snew[i] = c
        snew = "".join(snew)
        nsnew = [ len(c) for c in snew.split(".") if len(c) ]
        if all( [nn==n for nn,n in zip(nsnew,ns)] ):
            arrangements += 1
    return arrangements

def part1_bf(infile):
    sources = readInput12(infile)
    return sum([ count_arrangements_bf(s) for s in sources ])

In [128]:
print("Test 1:",part1_bf("examples/example12.txt"))
print("Part 1:",part1_bf("AOC2023inputs/input12.txt"))

Test 1: 21
Part 1: 7286


### Part 2

Brute forcing would obviously not work here...

Using recursion + memoization (crucial!), following explanation and tutorial at https://www.youtube.com/watch?v=g3Ms5e7Jdqo

In [181]:
cache = {}

def count_arrangements(pattern,numbers):

    # an empty pattern is only valid if there are no sources on its right 
    # (thus an empy numbers tuple) associated to it, otherwise it's an 
    # invalid pattern and it should not be counted
    if pattern=="":
        if numbers==():
            return 1
        else:
            return 0
    
    # if there are not more blocks expected but there's a # sign in the pattern
    # the configuration is invalid, otherwise there will only be ? in the pattern
    # and they should become .
    if numbers==():
        if '#' in pattern:
            return 0
        else:
            return 1
    
    # memoization
    key = (pattern,numbers)
    if key in cache.keys():
        return cache[key]

    arrangements = 0

    # if the first character in the pattern is either . or ? we recurse the counting
    # on the rest of the pattern
    if pattern[0] in ".?":
        arrangements += count_arrangements(pattern[1:],numbers)

    # if the first character in the pattern is either # or ? we need to check whether the block is valid 
    # and 3 conditions are satisfied, in which case we expand the pattern in subpatterns and recurse
    if pattern[0] in "#?":
        # the first spring count must be great or equal to the pattern lenght and
        # there must not be an operation spring in the pattern covered by the number count and
        # all spring are operationals
        if numbers[0] <= len(pattern) and \
            "." not in pattern[:numbers[0]] and \
            ( numbers[0] == len(pattern) or pattern[numbers[0]] != "#" ): 
            arrangements += count_arrangements(pattern[numbers[0]+1:],numbers[1:]) # if all conditions are satisfied, we develop the pattern with recursion

    cache[key] = arrangements
    return arrangements

def part1(infile):
    sources = readInput12(infile)
    return sum([ count_arrangements(s,ns) for s,ns in sources ])

print("Test 1:",part1("examples/example12.txt"))
print("Part 1:",part1("AOC2023inputs/input12.txt"))

Test 1: 21
Part 1: 7286


In [182]:
def part2(infile):
    sources = readInput12(infile)
    tot = 0
    for s,ns in sources:
        s5 = (5*(s+"?"))[:-1]
        ns5 = 5*ns
        tot += count_arrangements(s5,ns5) 
    return tot

In [183]:
print("Test 2:",part2("examples/example12.txt"))
print("Part 2:",part2("AOC2023inputs/input12.txt"))

Test 2: 525152
Part 2: 25470469710341
